## Creation of Conditions

Conditions are, next to variables, the central concept of this package. They define the necessary training of the model and connect the differential equation to the neural network.

The conditions can be imported from _torchphysics.problem.condition_.

There are essentally three different types of conditions:
* #### DiffEqCondition:
    These conditions handle the ODE/PDE that the model should solve. This means this condition somehow  
    implements an equation of the form:
    $$
        D(u, x, \lambda) = 0
    $$
    For $D$ any differential operator, $u$ the searched solution, $x$ some input variables from a   
    domain $\Omega$ and $\lambda$ some additional parameters (optional).
* #### BoundaryCondition: 
    This condition handles the boundary conditions of the problem and splits in three additional 
    classes:
    * #### DirichletCondition:

        For Dirichlet-conditions, e.g. $u(x) = g(x)$, on $\partial \Omega$.
    * #### NeumannCondition:
    
        For Neumann-conditions, e.g. $\vec{n} \nabla u(x) = g(x)$, on $\partial \Omega$.
    * #### DiffEqBoundaryCondition:

        For arbitrary boundary conditions.

* #### DataCondition:
    Mostly used for validation of the model and the inverse problem, where we want to find 
    parameters $\lambda$.

How to create some simple conditions will be shown next. To see all possible inputs and properties of the conditions, see the documentation of each class.

In the following, we create the conditions that belong to a simple Poisson-Problem:
$$
    \Delta u = -4.25\pi^2 \sin(\tfrac{\pi}{2} x_1)\cdot \cos(2\pi x_2), \text{ in } \Omega = [0, 1] \times [0, 1],
$$
$$
    u(x) = \sin(\tfrac{\pi}{2} x_1)\cdot \cos(2\pi x_2), \text{ on } \partial \Omega.
$$

The solution is easy to see: $u(x) = \sin(\tfrac{\pi}{2} x_1)\cdot \cos(2\pi x_2)$.

We start with the PDE. It is important that we bring the given problem in the form
$$
    D(u, x, \lambda) = 0
$$
so that the right hand side of the equation is zero! Here:
$$
    \Delta u + 4.25\pi^2 \sin(\tfrac{\pi}{2} x_1)\cdot \cos(2\pi x_2) = 0.
$$
First we start by defining a function, that implements the equation. This function needs to have two inputs, first the output of the neural network and second the data points (input of the network) as a dictionary. The Laplace-operator can be imported from utils.differentialoperators.

Also it is important, that the PDE-function uses only methods from PyTorch! While it is okay to use contsants from other libraries, e.g. $\pi$ from numpy.

In [1]:
from torchphysics.utils.differentialoperators import laplacian
import torch
import numpy as np

def pde_function(u, input):
    f = 4.25*np.pi*torch.sin(np.pi/2*input['x'][:, :1])*torch.cos(2*np.pi*input['x'][:, 1:])
    return laplacian(u, input['x']) + f

# the input dic. will always be in the form:
# input = {'Variable_name_1': data_points_of_variable,
#          'Variable_name_2': data_points_of_variable, ...}
# The exact creation of variables is shown in ...

# The data points will be always a tensor of the shape:
# (batch_size, dimension of variable)

Now connect this function with a DiffEqCondition:

In [2]:
from torchphysics.problem.condition import DiffEqCondition

pde_cond = DiffEqCondition(pde=pde_function, 
                           norm=torch.nn.MSELoss(), # norm for the loss
                           name='test_pde', # some arbitrary name (needs to be unique per condition)
                           sampling_strategy='grid', # The sampling strategy 
                           dataset_size=500) # number of datapoints for the training

We dont specify the domain in the condition! The domains are only connected to input variables of the domain. When a condition is connected/registered to a variable it will automatically know on which domain to sample. More on this in the file _variable_creation.ipynb_. 

Next we create the condition for the boundary. The structure is the same as before, first the function then connection to the condition-class. 

In [4]:
def bound_function(input): 
    return np.sin(np.pi/2*input['x'][:, :1])*np.cos(2*np.pi*input['x'][:, 1:])
# Since it is a Dirichlet-Condition, the function only gets the input data, not the
# model. 


from torchphysics.problem.condition import DirichletCondition

diri_cond = DirichletCondition(dirichlet_fun=bound_function,
                               name='test diri',
                               sampling_strategy='',
                               boundary_sampling_strategy='grid',
                               norm=torch.nn.MSELoss(),
                               weight=1.0,
                               dataset_size=200)
# The only difference to the DiffEqCondition is, that here we have to different 
# sampling strategies. One for the boundary and one for the additional variables.
# In this example there are no additional variables. But if we had, for example, 
# a time-variable, we could make grid sampling on the unit-square boundary and 
# sample at random points in time (sampling_strategy = 'random').    
